# Imports

In [1]:
import cv2
import pandas as pd
import dippykit as dip
import numpy as np                                   
import matplotlib.pyplot as plt   
from skimage import exposure 
from skimage.exposure import match_histograms 
import matlab.engine
import glob
import ssim.ssimlib as pyssim
from PIL import Image
import os 
import pywt
from keras.models import load_model
eng = matlab.engine.start_matlab()
eng.addpath("Function\InputWeights")
eng.addpath("Function\InputWeights_U")
eng.addpath("Function")

[2023-11-30 17:00:57,913] DEBUG: Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
[2023-11-30 17:00:58,260] DEBUG: Creating converter from 7 to 5
[2023-11-30 17:00:58,260] DEBUG: Creating converter from 5 to 7
[2023-11-30 17:00:58,260] DEBUG: Creating converter from 7 to 5
[2023-11-30 17:00:58,261] DEBUG: Creating converter from 5 to 7
[2023-11-30 17:01:01,188] WARNING: From c:\Users\ajayr\miniconda3\envs\dip_project\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



'C:\\Users\\ajayr\\Documents\\Masters\\3rd Sem\\Digital Image Processing\\Project\\Function\\InputWeights_U;C:\\Users\\ajayr\\Documents\\Masters\\3rd Sem\\Digital Image Processing\\Project\\Function\\InputWeights;C:\\Users\\ajayr\\Documents\\MATLAB;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addon_enable_disable_management\\matlab;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addon_updates\\matlab;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons\\cef;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons\\fileexchange;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons\\supportpackages;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons_common\\matlab;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons_install_location\\matlab;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons_product;C:\\Program Files\\MATLAB\\R2023a\\toolbox\\matlab\\addons_product_support\\matlab;C:\\P

# Functions

In [2]:
def nlmeans_denoise(image, h=3, templateWindowSize=7, searchWindowSize=21):
    # Perform Non-Local Means Denoising using OpenCV
    return cv2.fastNlMeansDenoising(image, None, h, templateWindowSize, searchWindowSize)

def wavelet_denoise(img, threshold=20):

    channels = cv2.split(img)

    # Apply wavelet denoising to each channel independently
    denoised_channels = []
    for channel in channels:
        # Perform 2D wavelet transform
        coeffs = pywt.dwt2(channel, 'bior1.3')

        # Thresholding: Set small coefficients to zero
        coeffs = tuple(map(lambda x: pywt.threshold(x, threshold, mode='soft'), coeffs))

        # Inverse 2D wavelet transform
        denoised_channel = pywt.idwt2(coeffs, 'bior1.3')

        # Clip values to be in the valid range [0, 255]
        denoised_channel = np.clip(denoised_channel, 0, 255)

        # Convert to uint8 (required for image display)
        denoised_channel = denoised_channel.astype(np.uint8)

        denoised_channels.append(denoised_channel)

    # Merge the denoised channels back into an RGB image
    denoised_img = cv2.merge(denoised_channels)

    return denoised_img

def combine_denoising_methods(image):

    # Apply NLMeans denoising
    nlmeans_denoised = nlmeans_denoise(image)

    # Apply wavelet denoising
    wavelet_denoised = wavelet_denoise(image)

    nlmeans_denoised = np.array(nlmeans_denoised)
    wavelet_denoised = np.array(wavelet_denoised)

    nlmeans_denoised_norm = nlmeans_denoised / 255.0
    wavelet_denoised_norm = wavelet_denoised / 255.0


    # Combine the results (you can experiment with different blending strategies)
    combined_denoised_norm = 0.2 * nlmeans_denoised_norm + 0.8 * wavelet_denoised_norm

    combined_denoised = (combined_denoised_norm * 255).astype(np.uint8)

    return combined_denoised

In [3]:
def psnr_2_lsts(lst1,gt_lst2):
    psnrs = []
    for i,j in zip(lst1,gt_lst2):
        val = cv2.PSNR(j, i)
        psnrs.append(val)
    return psnrs

def plot_normal_distribution(old_psnrs, new_psnrs):
    # Calculate mean and standard deviation for each dataset
    mean1 = np.mean(old_psnrs)
    std_dev1 = np.std(old_psnrs)

    mean2 = np.mean(new_psnrs)
    std_dev2 = np.std(new_psnrs)

    # Create a range of x values
    x = np.linspace(min(old_psnrs + new_psnrs), max(old_psnrs + new_psnrs), 100)

    # Generate probability density functions (PDFs) for each dataset
    pdf1 = 1 / (std_dev1 * np.sqrt(2 * np.pi)) * np.exp(-(x - mean1)**2 / (2 * std_dev1**2))
    pdf2 = 1 / (std_dev2 * np.sqrt(2 * np.pi)) * np.exp(-(x - mean2)**2 / (2 * std_dev2**2))

    # Plot the normal distribution curves
    plt.plot(x, pdf1, label='old_psnrs', color='blue')
    plt.plot(x, pdf2, label='new_psnrs', color='red')

    plt.xlabel('X-axis')
    plt.ylabel('Probability Density')
    plt.title('Normal Distribution of Data')
    plt.legend()

    plt.show()

In [4]:
class Evaluation_Metrics:
    def __init__(self):
        pass  

    def psnr(self, img1, img2):
        return dip.metrics.PSNR(img1, img2)

    def mse(self, img1, img2):
        return dip.metrics.MSE(img1, img2)

    def ssim(self, img1, img2):
        img1=cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
        img2=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
        return dip.metrics.SSIM(img1, img2)[0]
    def cwssim(self,img1,img2):
        img1_pillow = Image.fromarray(img1)
        img2_pillow = Image.fromarray(img2)
        similarity = pyssim.SSIM(img1_pillow).cw_ssim_value(img2_pillow)
        return similarity
    def unique(self, img1, img2):
        return eng.mslUNIQUE(img1, img2)

    def ms_unique(self, img1, img2):
        return eng.mslMSUNIQUE(img1, img2)

    def summer(self, img1, img2):
        return eng.SUMMER(img1, img2)

    def csv(self, img1, img2):
        return eng.csv(img1, img2)
metrics=Evaluation_Metrics()

In [5]:
def evaluation_metrics(noise_type,noise_level,denoisy_img,noisy_img,gt):
    eval_dict = {}
    eval_dict['noise_type'] = noise_type
    eval_dict['noise_level'] = noise_level
    # ******************* PSNR *******************
    psnr_noisy = metrics.psnr(noisy_img,gt)
    psnr_denoisy =metrics.psnr(denoisy_img,gt)
    eval_dict['PSNR_noisy'] = psnr_noisy
    eval_dict['PSNR_denoisy'] = psnr_denoisy

    # ******************* SSIM ******************* 
    ssim_noisy = metrics.ssim(noisy_img,gt)
    ssim_denoisy =metrics.ssim(denoisy_img,gt)
    eval_dict['SSIM_noisy'] = ssim_noisy
    eval_dict['SSIM_denoisy'] = ssim_denoisy

    # ******************* CW-SSIM ******************* 
    cw_ssim_noisy = metrics.cwssim(noisy_img,gt)
    cw_ssim_denoisy = metrics.cwssim(denoisy_img,gt)
    eval_dict['CW-SSIM_noisy'] = cw_ssim_noisy
    eval_dict['CW-SSIM_denoisy'] = ssim_denoisy
    # ******************* UNIQUE ******************* 
    unique_noisy = metrics.unique(noisy_img,gt)
    unique_denoisy =metrics.unique(denoisy_img,gt)
    eval_dict['unique_noisy'] = unique_noisy
    eval_dict['unique_denoisy'] = unique_denoisy

    # ******************* MS-UNIQUE ******************* 
    ms_unique_noisy = metrics.ms_unique(noisy_img,gt)
    ms_unique_denoisy =metrics.ms_unique(denoisy_img,gt)
    eval_dict['ms_unique_noisy'] = ms_unique_noisy
    eval_dict['ms_unique_denoisy'] = ms_unique_denoisy

    # ******************* CSV ******************* 
    csv_noisy = metrics.csv(noisy_img,gt)
    csv_denoisy =metrics.csv(denoisy_img,gt)
    eval_dict['csv_noisy'] = csv_noisy
    eval_dict['csv_denoisy'] = csv_denoisy
    # ******************* SUMMER *******************
    summer_noisy = metrics.summer(noisy_img,gt)
    summer_denoisy =metrics.summer(denoisy_img,gt)
    eval_dict['summer_noisy'] = summer_noisy
    eval_dict['summer_denoisy'] = summer_denoisy

    return eval_dict

def list_of_dicts_to_dataframe(data):
    df = pd.DataFrame(data)
    return df

In [6]:
def convert(img):
    image= cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    return image

# Load Datasets

In [7]:
img_folder=glob.glob('CURE-OR-Train/*')

In [8]:
color_image_folder=img_folder[:4950]

In [9]:
categories = {
    "01_No_challenge": color_image_folder[:150],
    "02_Resize_level1": color_image_folder[150:300],
    "02_Resize_level2": color_image_folder[300:450],
    "02_Resize_level3": color_image_folder[450:600],
    "02_Resize_level4": color_image_folder[600:750],
    "03_Underexposure_level1": color_image_folder[750:900],
    "03_Underexposure_level2": color_image_folder[900:1050],
    "03_Underexposure_level3": color_image_folder[1050:1200],
    "03_Underexposure_level4": color_image_folder[1200:1350],
    "04_Overexposure_level1": color_image_folder[1350:1500],
    "04_Overexposure_level2": color_image_folder[1500:1650],
    "04_Overexposure_level3": color_image_folder[1650:1800],
    "04_Overexposure_level4": color_image_folder[1800:1950],
    "05_Gaussian_blur_level1": color_image_folder[1950:2100],
    "05_Gaussian_blur_level2": color_image_folder[2100:2250],
    "05_Gaussian_blur_level3": color_image_folder[2250:2400],
    "05_Gaussian_blur_level4": color_image_folder[2400:2550],
    "06_Contrast_level1": color_image_folder[2550:2700],
    "06_Contrast_level2": color_image_folder[2700:2850],
    "06_Contrast_level3": color_image_folder[2850:3000],
    "06_Contrast_level4": color_image_folder[3000:3150],
    "07_Dirty_lens_1_level1": color_image_folder[3150:3300],
    "07_Dirty_lens_1_level2": color_image_folder[3300:3450],
    "07_Dirty_lens_1_level3": color_image_folder[3450:3600],
    "07_Dirty_lens_1_level4": color_image_folder[3600:3750],
    "08_Dirty_lens_2_level1": color_image_folder[3750:3900],
    "08_Dirty_lens_2_level2": color_image_folder[3900:4050],
    "08_Dirty_lens_2_level3": color_image_folder[4050:4200],
    "08_Dirty_lens_2_level4": color_image_folder[4200:4350],
    "09_Salt_and_pepper_noise_level1": color_image_folder[4350:4500],
    "09_Salt_and_pepper_noise_level2": color_image_folder[4500:4650],
    "09_Salt_and_pepper_noise_level3": color_image_folder[4650:4800],
    "09_Salt_and_pepper_noise_level4": color_image_folder[4800:4950]
}

In [10]:
reference_bg1 = r"C:\Users\ajayr\Documents\Masters\3rd Sem\Digital Image Processing\Project\CURE-OR-Train\00022.jpg"

# Denoising Algorithm NLMean + Wavelet Denoising

In [11]:
empty=[]
h = 10 
tempWinSize = 7
searchWinSize = 21

## Resize


##### Level 1

In [12]:
for i in range(10):
    noisy_image=cv2.imread(categories['02_Resize_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Resize","1",denoised_img,noisy_image,reference_image))

##### Level 2

In [13]:
for i in range(10):
    noisy_image=cv2.imread(categories['02_Resize_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Resize","2",denoised_img,noisy_image,reference_image))

##### Level 3

In [14]:
for i in range(10):
    noisy_image=cv2.imread(categories['02_Resize_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Resize","3",denoised_img,noisy_image,reference_image))

##### Level 4

In [15]:
for i in range(10):
    noisy_image=cv2.imread(categories['02_Resize_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Resize","4",denoised_img,noisy_image,reference_image))

## Underexposure

##### Level 1

In [16]:
for i in range(10):
    noisy_image=cv2.imread(categories['03_Underexposure_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Underexposure","1",denoised_img,noisy_image,reference_image))

##### Level 2

In [17]:
for i in range(10):
    noisy_image=cv2.imread(categories['03_Underexposure_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Underexposure","2",denoised_img,noisy_image,reference_image))

##### Level 3

In [18]:
for i in range(10):
    noisy_image=cv2.imread(categories['03_Underexposure_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Underexposure","3",denoised_img,noisy_image,reference_image))

##### Level 4

In [19]:
for i in range(10):
    noisy_image=cv2.imread(categories['03_Underexposure_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Underexposure","4",denoised_img,noisy_image,reference_image))

## Overexposure

##### Level 1

In [20]:
for i in range(10):
    noisy_image=cv2.imread(categories['04_Overexposure_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Overexposure","1",denoised_img,noisy_image,reference_image))

##### Level 2

In [21]:
for i in range(10):
    noisy_image=cv2.imread(categories['04_Overexposure_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Overexposure","2",denoised_img,noisy_image,reference_image))

##### Level 3

In [22]:
for i in range(10):
    noisy_image=cv2.imread(categories['04_Overexposure_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Overexposure","3",denoised_img,noisy_image,reference_image))

##### Level 4

In [23]:
for i in range(10):
    noisy_image=cv2.imread(categories['04_Overexposure_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Overexposure","4",denoised_img,noisy_image,reference_image))

## Gaussian blur

##### Level 1

In [24]:
for i in range(10):
    noisy_image=cv2.imread(categories['05_Gaussian_blur_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Gaussian Blur","1",denoised_img,noisy_image,reference_image))

##### Level 2

In [25]:
for i in range(10):
    noisy_image=cv2.imread(categories['05_Gaussian_blur_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Gaussian Blur","2",denoised_img,noisy_image,reference_image))

##### Level 3

In [26]:
for i in range(10):
    noisy_image=cv2.imread(categories['05_Gaussian_blur_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Gaussian Blur","3",denoised_img,noisy_image,reference_image))

##### Level 4

In [27]:
for i in range(10):
    noisy_image=cv2.imread(categories['05_Gaussian_blur_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Gaussian Blur","4",denoised_img,noisy_image,reference_image))

## Contrast level

##### Level 1

In [28]:
for i in range(10):
    noisy_image=cv2.imread(categories['06_Contrast_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Contrast level","1",denoised_img,noisy_image,reference_image))

##### Level 2

In [29]:
for i in range(10):
    noisy_image=cv2.imread(categories['06_Contrast_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Contrast level","2",denoised_img,noisy_image,reference_image))

##### Level 3

In [30]:
for i in range(10):
    noisy_image=cv2.imread(categories['06_Contrast_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Contrast level","3",denoised_img,noisy_image,reference_image))

##### Level 4

In [31]:
for i in range(10):
    noisy_image=cv2.imread(categories['06_Contrast_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Contrast level","4",denoised_img,noisy_image,reference_image))

## Dirty Lens 1

##### Level 1

In [32]:
for i in range(10):
    noisy_image=cv2.imread(categories['07_Dirty_lens_1_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Diry Lens 1","1",denoised_img,noisy_image,reference_image))

##### Level 2

In [33]:
for i in range(10):
    noisy_image=cv2.imread(categories['07_Dirty_lens_1_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Diry Lens 1","2",denoised_img,noisy_image,reference_image))

##### Level 3

In [34]:
for i in range(10):
    noisy_image=cv2.imread(categories['07_Dirty_lens_1_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Diry Lens 1","3",denoised_img,noisy_image,reference_image))

##### Level 4

In [35]:
for i in range(10):
    noisy_image=cv2.imread(categories['07_Dirty_lens_1_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Diry Lens 1","4",denoised_img,noisy_image,reference_image))

## Dirty Lens 2

##### Level 1

In [36]:
for i in range(10):
    noisy_image=cv2.imread(categories['08_Dirty_lens_2_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Diry Lens 2","1",denoised_img,noisy_image,reference_image))

##### Level 2

In [37]:
for i in range(10):
    noisy_image=cv2.imread(categories['08_Dirty_lens_2_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Diry Lens 2","2",denoised_img,noisy_image,reference_image))

##### Level 3

In [38]:
for i in range(10):
    noisy_image=cv2.imread(categories['08_Dirty_lens_2_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Diry Lens 2","3",denoised_img,noisy_image,reference_image))

##### Level 4

In [39]:
for i in range(10):
    noisy_image=cv2.imread(categories['08_Dirty_lens_2_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Diry Lens 2","4",denoised_img,noisy_image,reference_image))

## Salt and Pepper

##### Level 1

In [40]:
for i in range(10):
    noisy_image=cv2.imread(categories['09_Salt_and_pepper_noise_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Salt and Pepper","1",denoised_img,noisy_image,reference_image))

##### Level 2

In [41]:
for i in range(10):
    noisy_image=cv2.imread(categories['09_Salt_and_pepper_noise_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Salt and Pepper","2",denoised_img,noisy_image,reference_image))

##### Level 3

In [42]:
for i in range(10):
    noisy_image=cv2.imread(categories['09_Salt_and_pepper_noise_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Salt and Pepper","3",denoised_img,noisy_image,reference_image))

##### Level 4

In [43]:
for i in range(10):
    noisy_image=cv2.imread(categories['09_Salt_and_pepper_noise_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = combine_denoising_methods(noisy_image)
    empty.append(evaluation_metrics("Salt and Pepper","4",denoised_img,noisy_image,reference_image))

In [44]:
Final_Data=list_of_dicts_to_dataframe(empty)
Final_Data


,noise_type,noise_level,PSNR_noisy,PSNR_denoisy,SSIM_noisy,SSIM_denoisy,CW-SSIM_noisy,CW-SSIM_denoisy,unique_noisy,unique_denoisy,ms_unique_noisy,ms_unique_denoisy,csv_noisy,csv_denoisy,summer_noisy,summer_denoisy
0,Resize,1,49.495453,29.419853,0.991943,0.979586,0.982948,0.979586,0.988335,0.986372,0.989336,0.987236,0.993298,0.990115,4.908180,4.831170
1,Resize,1,48.099791,29.362503,0.992052,0.982478,0.990709,0.982478,0.989720,0.987873,0.990120,0.988823,0.993959,0.990351,4.892379,4.801595
2,Resize,1,49.085993,29.375618,0.991164,0.981812,0.983002,0.981812,0.988041,0.985710,0.987315,0.985726,0.997866,0.990089,4.908367,4.835393
3,Resize,1,48.790185,29.405195,0.992166,0.965187,0.992644,0.965187,0.990173,0.987180,0.990856,0.988732,0.993649,0.988927,4.903935,4.807363
4,Resize,1,48.159459,29.389947,0.992729,0.982655,0.989717,0.982655,0.989651,0.987181,0.990568,0.988728,0.993837,0.989601,4.897213,4.799509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Salt and Pepper,4,27.883859,27.964642,0.031104,0.044517,0.270894,0.044517,0.001134,0.001144,0.102823,0.103644,0.954979,0.954042,2.468916,2.466521
316,Salt and Pepper,4,27.903623,27.976514,0.031429,0.044995,0.285287,0.044995,0.001630,0.001655,0.096972,0.097954,0.956101,0.957058,2.876035,2.872791
317,Salt and Pepper,4,27.893378,27.945950,0.036818,0.051181,0.290597,0.051181,0.001997,0.002018,0.121939,0.122833,0.953420,0.953982,3.297429,3.286162
318,Salt and Pepper,4,27.885621,27.968331,0.031711,0.045535,0.283078,0.045535,0.001063,0.001082,0.093628,0.094379,0.956077,0.955465,2.118243,2.112244


In [45]:
Final_Data.to_csv(r"C:\Users\ajayr\Documents\Masters\3rd Sem\Digital Image Processing\Project\Results\Denoising\Wavelet+NLMeans\CURE-OR\CURE_OR_results.csv")

# Denoising Algorithm Autoencoders

In [46]:
empty=[]
model_path = 'Models\model_CURE_OR.h5'
model = load_model(model_path)

[2023-11-30 17:08:15,730] DEBUG: Creating converter from 3 to 5
[2023-11-30 17:08:15,857] WARNING: From c:\Users\ajayr\miniconda3\envs\dip_project\lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.

[2023-11-30 17:08:15,900] WARNING: From c:\Users\ajayr\miniconda3\envs\dip_project\lib\site-packages\keras\src\layers\pooling\max_pooling2d.py:161: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



## Resize

##### Level 1

In [47]:
for i in range(10):
    noisy_image=cv2.imread(categories['02_Resize_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Resize","1",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 114ms/step


##### Level 2

In [48]:
for i in range(10):
    noisy_image=cv2.imread(categories['02_Resize_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Resize","2",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 113ms/step


##### Level 3

In [49]:
for i in range(10):
    noisy_image=cv2.imread(categories['02_Resize_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Resize","3",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 107ms/step


##### Level 4

In [50]:
for i in range(10):
    noisy_image=cv2.imread(categories['02_Resize_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Resize","4",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 116ms/step


## Underexposure

##### Level 1

In [51]:
for i in range(10):
    noisy_image=cv2.imread(categories['03_Underexposure_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Underexposure","1",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 122ms/step


##### Level 2

In [52]:
for i in range(10):
    noisy_image=cv2.imread(categories['03_Underexposure_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Underexposure","2",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 119ms/step


##### Level 3

In [53]:
for i in range(10):
    noisy_image=cv2.imread(categories['03_Underexposure_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Underexposure","3",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 112ms/step


##### Level 4

In [54]:
for i in range(10):
    noisy_image=cv2.imread(categories['03_Underexposure_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Underexposure","4",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 112ms/step


## Overexposure

##### Level 1

In [55]:
for i in range(10):
    noisy_image=cv2.imread(categories['04_Overexposure_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Overexposure","1",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 116ms/step


##### Level 2

In [56]:
for i in range(10):
    noisy_image=cv2.imread(categories['04_Overexposure_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Overexposure","2",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 113ms/step


##### Level 3

In [57]:
for i in range(10):
    noisy_image=cv2.imread(categories['04_Overexposure_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Overexposure","3",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 111ms/step


##### Level 4

In [58]:
for i in range(10):
    noisy_image=cv2.imread(categories['04_Overexposure_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Overexposure","4",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 112ms/step


## Gaussian blur

##### Level 1

In [59]:
for i in range(10):
    noisy_image=cv2.imread(categories['05_Gaussian_blur_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Gaussian_blur","1",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 114ms/step


##### Level 2

In [60]:
for i in range(10):
    noisy_image=cv2.imread(categories['05_Gaussian_blur_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Gaussian_blur","2",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 110ms/step


##### Level 3

In [61]:
for i in range(10):
    noisy_image=cv2.imread(categories['05_Gaussian_blur_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Gaussian_blur","3",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 118ms/step


##### Level 4

In [62]:
for i in range(10):
    noisy_image=cv2.imread(categories['05_Gaussian_blur_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Gaussian_blur","4",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 104ms/step


## Contrast

##### Level 1

In [63]:
for i in range(10):
    noisy_image=cv2.imread(categories['06_Contrast_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Contrast Level","1",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 112ms/step


##### Level 2

In [64]:
for i in range(10):
    noisy_image=cv2.imread(categories['06_Contrast_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Contrast Level","2",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 122ms/step


##### Level 3

In [65]:
for i in range(10):
    noisy_image=cv2.imread(categories['06_Contrast_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Contrast Level","3",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 111ms/step


##### Level 4

In [66]:
for i in range(10):
    noisy_image=cv2.imread(categories['06_Contrast_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Contrast Level","4",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 116ms/step


## Dirty lens 1

##### Level 1

In [67]:
for i in range(10):
    noisy_image=cv2.imread(categories['07_Dirty_lens_1_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Dirty Lens 1","1",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 111ms/step


##### Level 2

In [68]:
for i in range(10):
    noisy_image=cv2.imread(categories['07_Dirty_lens_1_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Dirty Lens 1","2",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 113ms/step


##### Level 3

In [69]:
for i in range(10):
    noisy_image=cv2.imread(categories['07_Dirty_lens_1_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Dirty Lens 1","3",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 109ms/step


##### Level 4

In [70]:
for i in range(10):
    noisy_image=cv2.imread(categories['07_Dirty_lens_1_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Dirty Lens 1","4",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 113ms/step


## Dirty Lens 2

##### Level 1

In [71]:
for i in range(10):
    noisy_image=cv2.imread(categories['08_Dirty_lens_2_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Dirty Lens 2","1",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 115ms/step


##### Level 2

In [72]:
for i in range(10):
    noisy_image=cv2.imread(categories['08_Dirty_lens_2_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Dirty Lens 2","2",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 119ms/step


##### Level 3

In [73]:
for i in range(10):
    noisy_image=cv2.imread(categories['08_Dirty_lens_2_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Dirty Lens 2","3",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 110ms/step


##### Level 4

In [74]:
for i in range(10):
    noisy_image=cv2.imread(categories['08_Dirty_lens_2_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Dirty Lens 2","4",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 121ms/step


## Salt and Pepper

##### Level 1

In [75]:
for i in range(10):
    noisy_image=cv2.imread(categories['09_Salt_and_pepper_noise_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Salt and Pepper","1",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 123ms/step


##### Level 2

In [76]:
for i in range(10):
    noisy_image=cv2.imread(categories['09_Salt_and_pepper_noise_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Salt and Pepper","2",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 117ms/step


##### Level 3

In [77]:
for i in range(10):
    noisy_image=cv2.imread(categories['09_Salt_and_pepper_noise_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Salt and Pepper","3",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 115ms/step


##### Level 4

In [78]:
for i in range(10):
    noisy_image=cv2.imread(categories['09_Salt_and_pepper_noise_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    img=np.reshape(noisy_image, (1, 256, 256, 3))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = model.predict(img)
    denoised_img=np.reshape(denoised_img, (256, 256, 3))
    denoised_img = (denoised_img * 255).astype(np.uint8)
    empty.append(evaluation_metrics("Salt and Pepper","4",denoised_img,noisy_image,reference_image))

1/1 [==============================] - 0s 111ms/step


In [79]:
Final_Data=list_of_dicts_to_dataframe(empty)
Final_Data

,noise_type,noise_level,PSNR_noisy,PSNR_denoisy,SSIM_noisy,SSIM_denoisy,CW-SSIM_noisy,CW-SSIM_denoisy,unique_noisy,unique_denoisy,ms_unique_noisy,ms_unique_denoisy,csv_noisy,csv_denoisy,summer_noisy,summer_denoisy
0,Resize,1,49.495453,28.932970,0.991943,0.770124,0.982948,0.770124,0.988335,0.000718,0.989336,0.262296,0.993298,0.965931,4.908180,1.383874
1,Resize,1,48.099791,28.507022,0.992052,0.724948,0.990709,0.724948,0.989720,0.002527,0.990120,0.231178,0.993959,0.957071,4.892379,4.999045
2,Resize,1,49.085993,29.115937,0.991164,0.725787,0.983002,0.725787,0.988041,0.000072,0.987315,0.144336,0.997866,0.979053,4.908367,4.999834
3,Resize,1,48.790185,29.197247,0.992166,0.723405,0.992644,0.723405,0.990173,0.027565,0.990856,0.372651,0.993649,0.949613,4.903935,1.233038
4,Resize,1,48.159459,28.679472,0.992729,0.756882,0.989717,0.756882,0.989651,0.000609,0.990568,0.148970,0.993837,0.954681,4.897213,3.832468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Salt and Pepper,4,27.883859,28.989451,0.031104,0.772142,0.270894,0.772142,0.001134,0.000116,0.102823,0.193195,0.954979,0.971632,2.468916,5.000000
316,Salt and Pepper,4,27.903623,28.790719,0.031429,0.766909,0.285287,0.766909,0.001630,0.000089,0.096972,0.152551,0.956101,0.971414,2.876035,5.000000
317,Salt and Pepper,4,27.893378,29.629162,0.036818,0.743835,0.290597,0.743835,0.001997,0.000144,0.121939,0.184065,0.953420,0.967775,3.297429,5.000000
318,Salt and Pepper,4,27.885621,28.611380,0.031711,0.753719,0.283078,0.753719,0.001063,0.000100,0.093628,0.149705,0.956077,0.971228,2.118243,5.000000


In [80]:
Final_Data.to_csv(r"C:\Users\ajayr\Documents\Masters\3rd Sem\Digital Image Processing\Project\Results\Denoising\Deep learning\CURE_OR\CURE_OR_results.csv")

# Enhancement Algorithm Histogram Matching

## Resize

In [81]:
empty=[]

##### Level 1

In [82]:
for i in range(10):
    noisy_image=cv2.imread(categories['02_Resize_level1'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Resize","1",denoised_img,noisy_image,reference_image))

##### Level 2

In [83]:
for i in range(10):
    noisy_image=cv2.imread(categories['02_Resize_level2'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Resize","2",denoised_img,noisy_image,reference_image))

##### Level 3

In [84]:
for i in range(10):
    noisy_image=cv2.imread(categories['02_Resize_level3'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Resize","3",denoised_img,noisy_image,reference_image))

##### Level 4

In [85]:
for i in range(10):
    noisy_image=cv2.imread(categories['02_Resize_level4'][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Resize","4",denoised_img,noisy_image,reference_image))

## Underexposure

##### Level 1

In [86]:
for i in range(10):
    noisy_image=cv2.imread(categories["03_Underexposure_level1"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Underexposure","1",denoised_img,noisy_image,reference_image))

##### Level 2

In [87]:
for i in range(10):
    noisy_image=cv2.imread(categories["03_Underexposure_level2"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Underexposure","2",denoised_img,noisy_image,reference_image))

##### Level 3

In [88]:
for i in range(10):
    noisy_image=cv2.imread(categories["03_Underexposure_level3"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Underexposure","3",denoised_img,noisy_image,reference_image))

##### Level 4

In [89]:
for i in range(10):
    noisy_image=cv2.imread(categories["03_Underexposure_level4"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Underexposure","4",denoised_img,noisy_image,reference_image))

## Overexposure

##### Level 1

In [90]:
for i in range(10):
    noisy_image=cv2.imread(categories["04_Overexposure_level1"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Overexposure","1",denoised_img,noisy_image,reference_image))

##### Level 2

In [91]:
for i in range(10):
    noisy_image=cv2.imread(categories["04_Overexposure_level2"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Overexposure","2",denoised_img,noisy_image,reference_image))

##### Level 3

In [92]:
for i in range(10):
    noisy_image=cv2.imread(categories["04_Overexposure_level3"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Overexposure","3",denoised_img,noisy_image,reference_image))

##### Level 4

In [93]:
for i in range(10):
    noisy_image=cv2.imread(categories["04_Overexposure_level4"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Overexposure","4",denoised_img,noisy_image,reference_image))

## Gaussian blur

##### Level 1

In [94]:
for i in range(10):
    noisy_image=cv2.imread(categories["05_Gaussian_blur_level1"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Gaussian_blur","1",denoised_img,noisy_image,reference_image))

##### Level 2

In [95]:
for i in range(10):
    noisy_image=cv2.imread(categories["05_Gaussian_blur_level2"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Gaussian_blur","2",denoised_img,noisy_image,reference_image))

##### Level 3

In [96]:
for i in range(10):
    noisy_image=cv2.imread(categories["05_Gaussian_blur_level3"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Gaussian_blur","3",denoised_img,noisy_image,reference_image))

##### Level 4

In [97]:
for i in range(10):
    noisy_image=cv2.imread(categories["05_Gaussian_blur_level4"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Gaussian_blur","4",denoised_img,noisy_image,reference_image))

## Contrast

##### Level 1

In [98]:
for i in range(10):
    noisy_image=cv2.imread(categories["06_Contrast_level1"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Contrast","1",denoised_img,noisy_image,reference_image))

##### Level 2

In [99]:
for i in range(10):
    noisy_image=cv2.imread(categories["06_Contrast_level2"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Contrast","2",denoised_img,noisy_image,reference_image))

##### Level 3

In [100]:
for i in range(10):
    noisy_image=cv2.imread(categories["06_Contrast_level3"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Contrast","3",denoised_img,noisy_image,reference_image))

##### Level 4

In [101]:
for i in range(10):
    noisy_image=cv2.imread(categories["06_Contrast_level4"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Contrast","4",denoised_img,noisy_image,reference_image))

## Dirty lens 1

##### Level 1

In [102]:
for i in range(10):
    noisy_image=cv2.imread(categories["07_Dirty_lens_1_level1"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Dirty Lens 1","1",denoised_img,noisy_image,reference_image))

##### Level 2

In [103]:
for i in range(10):
    noisy_image=cv2.imread(categories["07_Dirty_lens_1_level2"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Dirty Lens 1","2",denoised_img,noisy_image,reference_image))

##### Level 3

In [104]:
for i in range(10):
    noisy_image=cv2.imread(categories["07_Dirty_lens_1_level3"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Dirty Lens 1","3",denoised_img,noisy_image,reference_image))

##### Level 4

In [105]:
for i in range(10):
    noisy_image=cv2.imread(categories["07_Dirty_lens_1_level4"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Dirty Lens 1","4",denoised_img,noisy_image,reference_image))

## Dirty lens 2

##### Level 1

In [106]:
for i in range(10):
    noisy_image=cv2.imread(categories["08_Dirty_lens_2_level1"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Dirty Lens 2","1",denoised_img,noisy_image,reference_image))

##### Level 2

In [107]:
for i in range(10):
    noisy_image=cv2.imread(categories["08_Dirty_lens_2_level2"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Dirty Lens 2","2",denoised_img,noisy_image,reference_image))

##### Level 3

In [108]:
for i in range(10):
    noisy_image=cv2.imread(categories["08_Dirty_lens_2_level3"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Dirty Lens 2","3",denoised_img,noisy_image,reference_image))

##### Level 4

In [109]:
for i in range(10):
    noisy_image=cv2.imread(categories["08_Dirty_lens_2_level4"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Dirty Lens 2","4",denoised_img,noisy_image,reference_image))

## Salt & pepper noise

##### Level 1

In [110]:
for i in range(10):
    noisy_image=cv2.imread(categories["09_Salt_and_pepper_noise_level1"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Salt and Pepper","1",denoised_img,noisy_image,reference_image))

##### Level 2

In [111]:
for i in range(10):
    noisy_image=cv2.imread(categories["09_Salt_and_pepper_noise_level2"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Salt and Pepper","2",denoised_img,noisy_image,reference_image))

##### Level 3

In [112]:
for i in range(10):
    noisy_image=cv2.imread(categories["09_Salt_and_pepper_noise_level3"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Salt and Pepper","3",denoised_img,noisy_image,reference_image))

##### Level 4

In [113]:
for i in range(10):
    noisy_image=cv2.imread(categories["09_Salt_and_pepper_noise_level4"][i])
    noisy_image=cv2.resize(noisy_image, (256, 256))
    reference_image = cv2.imread(categories['01_No_challenge'][i])
    reference_image=cv2.resize(reference_image, (256, 256))
    denoised_img = match_histograms(noisy_image,reference_image, channel_axis=-1)
    empty.append(evaluation_metrics("Salt and Pepper","4",denoised_img,noisy_image,reference_image))

In [114]:
Final_Data=list_of_dicts_to_dataframe(empty)
Final_Data

,noise_type,noise_level,PSNR_noisy,PSNR_denoisy,SSIM_noisy,SSIM_denoisy,CW-SSIM_noisy,CW-SSIM_denoisy,unique_noisy,unique_denoisy,ms_unique_noisy,ms_unique_denoisy,csv_noisy,csv_denoisy,summer_noisy,summer_denoisy
0,Resize,1,49.495453,46.751071,0.991943,0.990378,0.982948,0.990378,0.988335,0.979370,0.989336,0.983762,0.993298,0.990424,4.908180,4.856787
1,Resize,1,48.099791,46.120692,0.992052,0.990187,0.990709,0.990187,0.989720,0.984415,0.990120,0.986318,0.993959,0.990456,4.892379,4.837076
2,Resize,1,49.085993,46.910706,0.991164,0.989463,0.983002,0.989463,0.988041,0.979786,0.987315,0.980805,0.997866,0.990608,4.908367,4.852881
3,Resize,1,48.790185,46.084372,0.992166,0.990310,0.992644,0.990310,0.990173,0.984924,0.990856,0.987976,0.993649,0.990459,4.903935,4.855071
4,Resize,1,48.159459,45.904451,0.992729,0.991462,0.989717,0.991462,0.989651,0.984111,0.990568,0.985955,0.993837,0.990832,4.897213,4.849249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Salt and Pepper,4,27.883859,29.117595,0.031104,0.208882,0.270894,0.208882,0.001134,0.003306,0.102823,0.271708,0.954979,0.960452,2.468916,4.862430
316,Salt and Pepper,4,27.903623,29.122024,0.031429,0.246420,0.285287,0.246420,0.001630,0.007015,0.096972,0.251967,0.956101,0.961471,2.876035,4.957414
317,Salt and Pepper,4,27.893378,29.080472,0.036818,0.118823,0.290597,0.118823,0.001997,0.003631,0.121939,0.256178,0.953420,0.956510,3.297429,4.725632
318,Salt and Pepper,4,27.885621,29.105886,0.031711,0.247557,0.283078,0.247557,0.001063,0.005511,0.093628,0.247792,0.956077,0.960341,2.118243,4.845338


In [115]:
Final_Data.to_csv(r"C:\Users\ajayr\Documents\Masters\3rd Sem\Digital Image Processing\Project\Results\Enhancement\CURE-OR\CURE_OR_results.csv")